# 分享模型

在 Hugging Face，我们相信公开分享知识和资源，能实现人工智能的普及化，让每个人都能受益。我们鼓励你将你的模型与社区分享，以帮助他人节省时间和精力。

要与社区共享模型，你需要在 huggingface.co 上拥有一个帐户。你还可以加入现有的组织或创建一个新的组织。

在本教程中，你将学习两种在 [Model Hub](https://huggingface.co/models) 上共享训练好的或微调的模型的方法：

- 通过编程将文件推送到 Hub。
- 使用 Web 界面将文件拖放到 Hub。

## 仓库功能

Model Hub 上的每个仓库都像是一个典型的GitHub仓库。我们的仓库提供版本控制、提交历史记录以及可视化差异的能力。

`Model Hub`的内置版本控制基于`git`和`git-lfs`。换句话说，你可以将一个模型视为一个仓库，从而实现更好的访问控制和可扩展性。版本控制允许使用修订方法来固定特定版本的模型，可以使用提交哈希值、标签或分支来标记。

因此，你可以通过revision参数加载特定的模型版本：

In [ ]:
model = AutoModel.from_pretrained(
    "julien-c/EsperBERTo-small", revision="v2.0.1"  # tag name, or branch name, or commit hash
)

文件也可以轻松地在仓库中编辑，你可以查看提交历史记录以及差异：

![vis_diff](../../resources/show/vis_diff.png)

## 设置

在将模型共享到Hub之前，你需要拥有Hugging Face的凭证。如果你有访问终端的权限，请在安装🤗 Transformers的虚拟环境中运行以下命令。这将在你的Hugging Face缓存文件夹（默认为`~/.cache/`）中存储你的`access token`：

In [ ]:
huggingface-cli login

如果你正在使用像`Jupyter`或`Colaboratory`这样的 notebook，请确保你已安装了[huggingface_hub](https://huggingface.co/docs/hub/models-adding-libraries)库。该库允许你以编程方式与Hub进行交互。

In [ ]:
pip install huggingface_hub

然后使用notebook_login登录到Hub，并按照这里的[链接](https://huggingface.co/settings/tokens)生成一个token进行登录：

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## 转换模型适用于所有框架

为确保你的模型可以被使用不同框架的人使用，我们建议你将PyTorch和TensorFlow checkpoints都转换并上传。如果你跳过此步骤，用户仍然可以从其他框架加载你的模型，但速度会变慢，因为🤗 Transformers需要实时转换checkpoints。

为另一个框架转换checkpoints很容易。确保你已安装PyTorch和TensorFlow（请参阅此处的安装说明），然后在其他框架中找到适合你任务的特定模型。

### PyTorch

指定`from_tf=True`将 checkpoint 从 TensorFlow 转换为 PyTorch。

In [ ]:
pt_model = DistilBertForSequenceClassification.from_pretrained("path/to/awesome-name-you-picked", from_tf=True)
pt_model.save_pretrained("path/to/awesome-name-you-picked")

### TensorFlow

指定`from_pt=True`将 checkpoint 从 PyTorch 转换为 TensorFlow 。

In [ ]:
tf_model = TFDistilBertForSequenceClassification.from_pretrained("path/to/awesome-name-you-picked", from_pt=True)

然后，你可以使用新的 checkpoint 保存你的新 TensorFlow 模型：

In [ ]:
tf_model.save_pretrained("path/to/awesome-name-you-picked")

### JAX

如果模型在Flax中可用，你还可以将 PyTorch checkpoint 转换为 Flax ：

In [ ]:
flax_model = FlaxDistilBertForSequenceClassification.from_pretrained(
    "path/to/awesome-name-you-picked", from_pt=True
)

## 在训练过程中推送模型

### PyTorch

将模型分享到Hub就像添加一个额外的参数或回调函数一样简单。请记住，在[微调教程](../tutorials/5_fine_tune_pretrained_model.ipynb)中，`TrainingArguments`类是你指定超参数和附加训练选项的地方。其中一项训练选项包括直接将模型推送到`Hub`的能力。在你的`TrainingArguments`中设置`push_to_hub=True`：

In [ ]:
training_args = TrainingArguments(output_dir="my-awesome-model", push_to_hub=True)

像往常一样将你的训练参数传递给`Trainer`：

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

在你微调完模型后，在`Trainer`上调用`push_to_hub()`将训练好的模型推送到Hub。🤗 Transformers 甚至会自动将训练超参数、训练结果和框架版本添加到你的模型名片中！

In [ ]:
trainer.push_to_hub()

### TensorFlow

使用[PushToHubCallback](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/keras_callbacks#transformers.PushToHubCallback)将模型分享到 Hub。在[PushToHubCallback](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/keras_callbacks#transformers.PushToHubCallback)函数中，添加以下内容：

- 一个用于存储模型的输出目录。
- 一个tokenizer。
- `hub_model_id`，即你的Hub用户名和模型名称。

In [ ]:
from transformers import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="./your_model_save_path", tokenizer=tokenizer, hub_model_id="your-username/my-awesome-model"
)

将回调函数添加到[fit](https://keras.io/api/models/model_training_apis/)中，然后🤗 Transformers 会将训练好的模型推送到 Hub：

## 使用 push_to_hub 功能

你可以直接在你的模型上调用`push_to_hub`来将其上传到Hub。

在`push_to_hub`中指定你的模型名称：

In [ ]:
pt_model.push_to_hub("my-awesome-model")

这会在你的用户名下创建一个名为`my-awesome-model`的仓库。用户现在可以使用`from_pretrained`函数加载你的模型：

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("your_username/my-awesome-model")

如果你属于一个组织，并希望将你的模型推送到组织名称下，只需将其添加到`repo_id`中：

In [ ]:
pt_model.push_to_hub("my-awesome-org/my-awesome-model")

`push_to_hub`函数还可以用于向模型仓库添加其他文件。例如，向模型仓库中添加一个`tokenizer`：

In [ ]:
tokenizer.push_to_hub("my-awesome-model")

或者，你可能想要将你的微调后的PyTorch模型的TensorFlow版本添加进去：

In [ ]:
tf_model.push_to_hub("my-awesome-model")

现在，当你导航到你的Hugging Face个人资料时，你应该看到你新创建的模型仓库。点击文件选项卡将显示你已上传到仓库的所有文件。

有关如何创建和上传文件到仓库的更多详细信息，请参考[Hub文档](https://huggingface.co/docs/huggingface_hub/v0.25.1/guides/upload)。

## 使用Web界面上传

喜欢无代码方法的用户可以通过 Hugging Face 的 Web 界面上传模型。访问`huggingface.co/new`创建一个新的仓库：

![new_model_repo](../../resources/show/new_model_repo.png)

从这里开始，添加一些关于你的模型的信息：

1. 选择仓库的所有者。这可以是你本人或者你所属的任何组织。
2. 为你的项目选择一个名称，该名称也将成为仓库的名称。
3. 选择你的模型是公开还是私有。
4. 指定你的模型的许可证使用情况。

现在点击文件选项卡，然后点击添加文件按钮将一个新文件上传到你的仓库。接着拖放一个文件进行上传，并添加提交信息。

![upload_file](../../resources/show/upload_file.png)

## 添加模型名片
为了确保用户了解你的模型的能力、限制、潜在偏差和伦理考虑，请在仓库中添加一个模型名片。模型名片在`README.md`文件中定义。你可以通过以下方式添加模型名片：

- 手动创建并上传一个`README.md`文件。
- 在你的模型仓库中点击编辑模型名片按钮。

可以参考DistilBert的[模型名片](https://huggingface.co/distilbert/distilbert-base-uncased)来了解模型名片应该包含的信息类型。有关你可以在`README.md`文件中控制的更多选项的细节，例如模型的碳足迹或小部件示例，请参考[文档](https://huggingface.co/docs/hub/model-cards)。